# Food Sales Prediction Project
The information use in this project corresponds to 2013 sales of **1559** products and was collected by data scientist at BigMart across 10 stores in different cities. Some features of each product have been defined.   
In order to help to the retailer to understand the product properties and the outlets that play a crucial role in the sales, **this project aims to predict the sales of food products in each outlet, it will try to understand the properties of products and outlets which play a key role in increasing sales.** 

In this section the data provided for training is cleaned.
<p><a href="https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/#ProblemStatement" target:"_blank">source: <i>Analytics Vidhya 2013-2022</i></a></p>
<table>
    <tbody>
        <tr>
            <td>Variable</td>
            <td >Description</td>
        </tr>
        <tr>
            <td >Item_Identifier</td>
            <td >Unique product ID</td>
        </tr>
        <tr>
            <td>Item_Weight</td>
            <td >Weight of product</td>
        </tr>
        <tr>
            <td>Item_Fat_Content</td>
            <td>Whether the product is low fat or not</td>
        </tr>
        <tr>
            <td>Item_Visibility</td>
            <td>The % of total display area of all products in a store allocated to the particular product</td>
        </tr>
        <tr>
            <td>Item_Type</td>
            <td>The category to which the product belongs</td>
        </tr>
        <tr>
            <td>Item_MRP</td>
            <td>Maximum Retail Price (list price) of the product</td>
        </tr>
        <tr>
            <td>Outlet_Identifier</td>
            <td>Unique store ID</td>
        </tr>
        <tr>
            <td>Outlet_Establishment_Year</td>
            <td>The year in which store was established</td>
        </tr>
        <tr>
            <td>Outlet_Size</td>
            <td>The size of the store in terms of ground area covered</td>
        </tr>
        <tr>
            <td>Outlet_Location_Type</td>
            <td>The type of city in which the store is located</td>
        </tr>
        <tr>
            <td>Outlet_Type</td>
            <td>Whether the outlet is just a grocery store or some sort of supermarket</td>
        </tr>
    </tbody>
</table>

In [ ]:
from sklearn.compose import make_column_selector
import numpy as np
import pandas as pd

## Charging Information

In [2]:
df = pd.read_csv("../data/raw/train_data.csv")
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Characterizing DataFrame

In [11]:
df.shape

(8523, 12)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


Observations: 
- There is consistency in the type of data in each column
- <p style="color:rgb(150,0,0);">There are missing data in the columns: Item_Weight - 1463 missing values, and Outlet_Size - 2410 missing values.</p>

## Data Cleaning

Let's check if there is duplicated info

### Remove Duplicate or Irrelevant Observations

In [13]:
df.duplicated().sum()

0

<p style="color:green;">There is not duplicated rows.</p>


### Irrelevant Observations

Since the year in which store was established can be categorized as an irrelevant feature in the sales prediction and in this column there is missing information this column will be delete.

In [4]:
df.drop(columns = ["Item_Weight","Outlet_Establishment_Year"], inplace=True)
df.head()

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,Regular,0.019278,Soft Drinks,48.2692,OUT018,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,Low Fat,0.016760,Meat,141.6180,OUT049,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,Low Fat,0.000000,Household,53.8614,OUT013,High,Tier 3,Supermarket Type1,994.7052


In [15]:
len(df["Item_Identifier"].unique()) # this will give us the number of products. As was said the numbero of products os 1559a

1559

Missing values in the column "Outlet_size" will be covered in the next section. The rest of the columns do not seem to present categorical inconsistencies. 

### Fixing Structural Errors

#### Type of Information in Each Column

In [46]:
df.dtypes

Item_Identifier          object
Item_Weight             float64
Item_Fat_Content         object
Item_Visibility         float64
Item_Type                object
Item_MRP                float64
Outlet_Identifier        object
Outlet_Size              object
Outlet_Location_Type     object
Outlet_Type              object
Item_Outlet_Sales       float64
dtype: object

#### Checking Features
Let's create a function to check the features in the dataset.

In [5]:
num_features = make_column_selector(dtype_include="number")(df) 
_ = df[num_features].apply(lambda x: print("\n",f"max_{x.name}: {x.max()} ----- min_{x.name}: {x.min()}"))


 max_Item_Visibility: 0.328390948 ----- min_Item_Visibility: 0.0

 max_Item_MRP: 266.8884 ----- min_Item_MRP: 31.29

 max_Item_Outlet_Sales: 13086.9648 ----- min_Item_Outlet_Sales: 33.29


In [6]:
cat_features = make_column_selector(dtype_include="object")(df)
_ = df[cat_features].apply(lambda x: print("\n", f"{x.name}: {x.unique()}"))


 Item_Identifier: ['FDA15' 'DRC01' 'FDN15' ... 'NCF55' 'NCW30' 'NCW05']

 Item_Fat_Content: ['Low Fat' 'Regular' 'low fat' 'LF' 'reg']

 Item_Type: ['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']

 Outlet_Identifier: ['OUT049' 'OUT018' 'OUT010' 'OUT013' 'OUT027' 'OUT045' 'OUT017' 'OUT046'
 'OUT035' 'OUT019']

 Outlet_Size: ['Medium' nan 'High' 'Small']

 Outlet_Location_Type: ['Tier 1' 'Tier 3' 'Tier 2']

 Outlet_Type: ['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']


<div style="background-color:rgb(150,50,50); padding:10px;">
    <p>There are some irregularities in the Item_Fat_Content; it can be deduced that</p>
    <ul>
        <li>LF = Low Fat</li>
        <li>low fat = Low Fat</li>
        <li>reg = Regular</li>
    </ul>
</div>

In [7]:
df["Item_Fat_Content"].replace(["LF","reg","low fat"],["Low Fat", "Regular","Low Fat"], inplace=True)
df["Item_Fat_Content"].unique()

array(['Low Fat', 'Regular'], dtype=object)

### Identifying and Handling Missing Data
As observed above, there are missing data in the columns Outlet_Size ans Item_Weight. Let's check the missing data in each column.

Lets see the characterize the column Item weight

The products presents similar features so let's use the method imputer from sklearn to handle missing values, But first let's split the dataset into testing and training to avoid data leak.

In [8]:
df[df["Outlet_Size"].isna()].shape

(2410, 10)

There are 2410 missing values in the "Outlet Size" columns. Since the outlet size can be considered as an important feature to take into account when doing predictions of sales it is a good idea group the information in order to find patterns that can be useful to make decisions about the missing information.

In [56]:
df.groupby(["Outlet_Type","Outlet_Location_Type","Outlet_Identifier"])["Outlet_Size"].unique()

Outlet_Type        Outlet_Location_Type  Outlet_Identifier
Grocery Store      Tier 1                OUT019                [Small]
                   Tier 3                OUT010                  [nan]
Supermarket Type1  Tier 1                OUT046                [Small]
                                         OUT049               [Medium]
                   Tier 2                OUT017                  [nan]
                                         OUT035                [Small]
                                         OUT045                  [nan]
                   Tier 3                OUT013                 [High]
Supermarket Type2  Tier 3                OUT018               [Medium]
Supermarket Type3  Tier 3                OUT027               [Medium]
Name: Outlet_Size, dtype: object

Having seen the structure of the information in the column 'outlet size', it is no possible to determine to which category belong the rows with missing values, so a new category in the outlet size feature will be created and the missing data will be replace by the label "Missing".

<p style="color:green;">Note that yet we modify the whole dataset there is no a data leak since we don't use any estimator </p>

In [9]:
df["Outlet_Size"] = df["Outlet_Size"].fillna("Missing")
df["Outlet_Size"].isna().sum()

0

In [69]:
X_train.replace()
median_imputer = SimpleImputer(strategy='mean')
median_imputer.fit(X_train[["Item_Weight"]])
X_train["Item_Weight"] = median_imputer.transform(X_train[["Item_Weight"]])

In [74]:
X_train["Item_Weight"].isna().sum()

0

In [75]:
X_train.groupby("Item_Type").agg({"Item_Weight":["min", "max", "mean"]})

Item_Weight                  
                              min    max       mean
Item_Type                                          
Baking Goods                4.880  20.85  12.548598
Breads                      4.635  20.85  11.754047
Breakfast                   6.425  21.10  12.839844
Canned                      4.615  21.35  12.555931
Dairy                       4.805  20.70  13.367924
Frozen Foods                4.555  20.85  12.848406
Fruits and Vegetables       5.460  21.35  13.060665
Hard Drinks                 4.610  19.70  11.768003
Health and Hygiene          5.175  21.25  13.039031
Household                   5.030  21.25  13.400862
Meat                        5.150  21.25  13.225468
Others                      5.500  20.50  13.713712
Seafood                     5.365  20.75  12.414791
Snack Foods                 5.095  21.25  12.941721
Soft Drinks                 4.590  20.75  11.965412
Starchy Foods               6.695  21.20  13.616864

### Dataset Validation

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Item_Identifier       8523 non-null   object 
 1   Item_Fat_Content      8523 non-null   object 
 2   Item_Visibility       8523 non-null   float64
 3   Item_Type             8523 non-null   object 
 4   Item_MRP              8523 non-null   float64
 5   Outlet_Identifier     8523 non-null   object 
 6   Outlet_Size           8523 non-null   object 
 7   Outlet_Location_Type  8523 non-null   object 
 8   Outlet_Type           8523 non-null   object 
 9   Item_Outlet_Sales     8523 non-null   float64
dtypes: float64(3), object(7)
memory usage: 666.0+ KB


The information is consistent and there is no missing information.

Let's generate a csv file to save the cleaning information

In [14]:
df.to_csv(path_or_buf="../data/cleaned/dataset_cleaned.csv", index=False )